In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import runpy

result = runpy.run_path('datasorting_orientation.py')

In [4]:
val_cat = result['val_cat']
data_cat = result['data_cat']
dur_cat = result['dur_cat']
del result

In [5]:
def generate_linear_model(rate_matrix, params, params0):
    """
    Generate a linear model of the rate matrix.

    Parameters:
    - rate_matrix (ndarray): Rate matrix (Trial x Time)
    - params (ndarray): Parameters (peak velocity, duration) shape (Trial x 2)
    - params0 (array-like): Grand average parameters [peak velocity, average velocity]

    Returns:
    - linmod (dict): Linear model structure containing:
        - wv0: reduced model velocity coefficients
        - wv: full model velocity coefficients
        - wr: full model duration coefficients
        - ssc: corrected PSTH for full model
        - ssc0: corrected PSTH for reduced model
        - v00: grand average peak velocity
        - r00: grand average average velocity
    """
    vpeak = np.abs(params[:, 0])  # peak velocity
    sdur = np.abs(params[:, 1])   # duration
    ss = rate_matrix              # rate matrix

    ss0 = np.nanmean(ss, axis=0)  # mean rate across trials
    dss = ss - ss0                # deviation from mean rate
    v0 = np.nanmean(vpeak)        # mean peak velocity
    r0 = np.nanmean(15. / sdur)   # mean average velocity

    dv = vpeak - v0
    dr = 10. / sdur - r0
    dz = np.stack((dv, dr), axis=1)  # shape (Trial, 2)

    # Full model coefficients
    cc = dz.T @ dz
    w12 = np.linalg.pinv(cc) @ dz.T @ dss  # shape (2, Time)

    wv = w12[0, :]  # velocity coefficients
    wr = w12[1, :]  # duration coefficients

    # Reduced model with only velocity
    wv0 = (dv.T @ dss) / (dv.T @ dv)  # shape (Time,)

    # Grand averages
    v00 = params0[0]
    r00 = params0[1]

    # Corrected PSTH for full model
    wc = (v00 - v0) * wv + (r00 - r0) * wr
    ssc = ss0 + wc

    # Corrected PSTH for reduced model
    wc0 = (v00 - v0) * wv0
    ssc0 = ss0 + wc0

    linmod = {
        'wv0': wv0,
        'wv': wv,
        'wr': wr,
        'ssc': ssc,
        'ssc0': ssc0,
        'v00': v00,
        'r00': r00
    }

    return linmod


In [30]:
peak = []
for a in range(np.shape(val_cat)[0]):
    M = []
    for ori in range(8):
        temp = val_cat[a][ori]
        M.append(np.max(temp,axis = 1))
        # peak[a][ori]  = np.max(temp,axis = 1)
    peak.append(M)

peak = np.array(peak, dtype = object)

In [40]:
ori_num = 0
pram = [peak[a][ori_num], dur_cat[a][ori_num]]

In [50]:
np.array(pram)

array([[514.39288199, 508.56822898, 463.14519163, 439.14897035,
        484.17282187, 478.7446606 , 471.61526318, 502.34695951,
        445.13477079, 457.41223524, 537.19383713, 525.11327121,
        425.47725087, 472.71584566, 401.08219217, 408.45759253,
        397.86904736, 485.30530747, 486.35017937, 434.55935335,
        476.74643039, 422.95505888, 460.24792429, 488.46931248,
        407.34931775, 461.63630776, 321.47146306, 491.78986958,
        374.1672588 , 411.37414955, 459.50973332, 423.36718637,
        479.27322069, 436.82742267, 447.90934939, 498.54563162,
        501.65157679, 441.2404466 , 506.45637689],
       [ 48.        ,  45.        ,  50.        ,  50.        ,
         47.        ,  49.        ,  50.        ,  49.        ,
         46.        ,  53.        ,  50.        ,  48.        ,
         52.        ,  43.        ,  57.        ,  51.        ,
         51.        ,  47.        ,  54.        ,  50.        ,
         50.        ,  51.        ,  52.        ,  47

In [52]:
a=0; ori_num=0

temp_data = data_cat[a][ori_num]
pram = np.array([peak[a][ori_num], dur_cat[a][ori_num]])
pram2= np.mean(pram, axis = 1)

In [55]:
temp_data.shape

(39, 600)

In [56]:
pram.shape

(2, 39)

In [ ]:
linmod = generate_linear_model(temp_data, pram, pram2)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 39 is different from 2)

In [ ]:
final = []
for ori_num in range(8):
    for a in range(np.shape(data_cat)[0]):
        temp_data = data_cat[a][ori_num]
        pram = np.array([peak[a][ori_num], dur_cat[a][ori_num]])
        pram2= np.mean(pram, axis = 1)
        
        linmod = generate_linear_model(temp_data, pram, pram2);
        final{a,ori_num}=linmod;

In [ ]:
final={};
for ori_num = 1:8;
    for a = 1:size(data_cat,1);
        temp_data = data_cat{a, ori_num};
        pram = [peak{a, ori_num},dur_cat{a, ori_num}];
        pram2= mean(pram);
        
        linmod = generate_linear_model(temp_data, pram, pram2);
        final{a,ori_num}=linmod;
    end
end; clear a ori_num

In [ ]:
%% plotting

plot_data={};
for ori_num=1:8;

    pp1=[];
    pp2=[];
    pp3=[];
    pp4=[];
    pp5=[];
    for a=1:size(final,1);
        temp=final{a, ori_num};
        temp1=temp.wv0;
        temp2=temp.wv;
        temp3=temp.wr;
        temp4=temp.ssc;
        temp5=temp.ssc0;

        pp1=[pp1;temp1;];
        pp2=[pp2;temp2;];
        pp3=[pp3;temp3;];
        pp4=[pp4;temp4;];
        pp5=[pp5;temp5;];
    end
    
    %smoothing
    zpp1=filter_matrix(pp1', 'sigma', 2)';
    zpp2=filter_matrix(pp2', 'sigma', 2)';
    zpp3=filter_matrix(pp3', 'sigma', 2)';
    zpp4=filter_matrix(pp4', 'sigma', 2)';
    zpp5=filter_matrix(pp5', 'sigma', 2)';

    plot_data{ori_num,1}=zpp1;
    plot_data{ori_num,2}=zpp2;
    plot_data{ori_num,3}=zpp3;
    plot_data{ori_num,4}=zpp4;
    plot_data{ori_num,5}=zpp5;
end


%% plotting

for b=1:5;
    figure;
    for a=1:8;
        subplot(1,8,a)
        imagesc(plot_data{a,b})
    end

    set(gcf,'OuterPosition', [3, 270, 2200, 200])
    tightfig
end


%%
figure
for ori_num=1:8;

    subplot(2,4,ori_num)

    set(gcf,'color','w')
    set(gca,'linewidth',2);

    mtemp=mean(plot_data{ori_num});
    semtemp = std(plot_data{ori_num})/sqrt(length(plot_data{ori_num}));
    time= -300:300-1;
    h= boundedline(time, mtemp, semtemp, 'alpha', 'cmap', [0 0.7 0], 'transparency', 0.3)

    h.LineWidth = 1;
    xlim([-300 300]); ylim([0.02 0.18]);
end

set(gcf,'OuterPosition', [3, 270, 2000, 800])
tightfig

% 
% % GLM regression
% Predictors(:,1)=(squareform(visMat))';
% Predictors(:,2)=(squareform(magMat))';
% 
% % normalize matrices
% Predictors_zscore(:,1)=cosmo_normalize(Predictors(:,1),'zscore');
% Predictors_zscore(:,2)=cosmo_normalize(Predictors(:,2),'zscore');
% Predictors_intercept=[ones(size(Predictors_zscore,1),1) Predictors_zscore];
% 
% % get betas
% betas = Predictors_zscore \ DM_timepoint_zscore;
% betas_intercept = Predictors_intercept \ DM_timepoint_zscore;
% 
% Betas_Vis(s,t)=betas_intercept(2); %first element is a constant, starting from 2 are predictor betas
% Betas_Mag(s,t)=betas_intercept(3);